In [4]:
import numpy as np
import random as rd
from mnist import MNIST as mn
import time

In [5]:
# Lookup table of activation functions
actLookup = {"sigmoid":lambda x: 1/(1 + np.exp(-1 * x)),
                         "linear":lambda x: x}

In [255]:
class layer(object):
    # A neural net layer with n neurons as defined by the user and an activation function for the layer
    
    def __init__(self, numInputs, layerSize, actFun = "sigmoid"):
        
        # Size of the layer and the number of inputs
        self.layerSize = layerSize
        self.actFun = actFun
        # Incorporating the bias neuron
        self.numInputs = numInputs + 1
        self.inputMatrix = None
        
        # Numpy matrix of weights
        self.weights = np.array(np.random.randn(self.layerSize,self.numInputs))/100
        
        # Set the backward propagation values for this layer
        self.delta = None
        
    # Defining the forward function to the layer
    def forward(self, inputMatrix):
        """Forward() forward propagates the inputs to a layer"""
        
        # Convert to numpy array if not passed as a numpy array
        if(not(isinstance(inputMatrix,np.ndarray))):
            inputMatrix = np.array(inputMatrix)
        
        rows, columns = inputMatrix.shape
        
        # Dot product of input matrix (with extra 1s for the bias neuron) with the weight matrix
        inputPadded = np.append(np.ones(rows).reshape(rows,1), inputMatrix, axis = 1)
        self.inputMatrix = inputPadded
        layerOutput = np.dot(inputPadded, self.weights.T)
        
        # Pass through activation function
        self.output = actLookup[self.actFun](layerOutput)
        return self.output
    
    # Defining the backward propagation function
    def backward(self, trainTarget, delta):
        """Backward takes the delta from next layer and passes it on the previous layer"""
        
        if(not(isinstance(trainTarget,np.ndarray))):
            trainTarget = np.array(trainTarget)
        
        # If delta is none then this layer is an output layere
        if(delta is None):
            
            rows, columns = trainTarget.shape
            errorTerm = np.subtract(trainTarget, self.output)
            if self.actFun == "sigmoid":
                outputInvert = np.array([(1. - op) for op in self.output])
                term1 = np.multiply(outputInvert, self.output)
                term2 = np.multiply(errorTerm, term1)
                self.delta  = term2
    

        # If delta has been passed on to this layer, then this layer is a hidden layer
        else:
            
            # Delta passed back is wkh * deltak for every batch instance
            # Delta for each batch element is sum by column of the delta matrix passed back
            # by the next layer
            # deltaTerm3 = delta.sum(axis = 1)
            deltaTerm3 = delta
            
            # note: need to add derivatives for other activation functions
            if self.actFun == "sigmoid":
                    
                outputInvert = np.array([(1. - op) for op in self.output])
                # Term1 represents o(1-o)
                term1 = np.multiply(outputInvert, self.output)
                self.delta = np.multiply(term1,deltaTerm3)
        
        # Delta to pass on to preceding layer has to be 
        # matrix multiplication of these values by the weight matrix
        weightMatrix = self.weights[:,1:]
        
        deltaBack = weightMatrix.T.dot(self.delta.T)
        return deltaBack.T
    
    def updateWeights(self, learnRate):
        """updateWeights() uses the inputs and the delta stored in each layer after forward
        and backward propagation to derive the weight update rule"""
        
        # The udpate rule for each element in the matrix is given by
        # wi = wi + sum_over_instances(delta for neuron * input i to neuron * learning rate)
        # Compute the weight updates for every data point and then add those updates
        rows,columns = self.inputMatrix.shape
        weightUpdates = np.dot(self.inputMatrix.T,self.delta).T * (1/np.float(rows))
        self.weights = np.add(self.weights,weightUpdates)

In [256]:
class neuralNet(object):
    
    """Neural net object is a combination of layer objects. Has 2 functions
    predict and backprop"""
    
    def __init__(self, layerList, actFun = "sigmoid"):
        
        """Takes in the layerlist as input and generates as many layers"""
        
        # Activations to be used in the net
        self.actFun = actFun
        
        # Creating the layers
        self.layers = [layer(numInputs=layerList[i - 1],
                             layerSize=layerList[i],                            
                             actFun=self.actFun) for i in range(1, len(layerList))]
        
    def getWeights(self):
        return [layer.weights for layer in self.layers]
        
    def predict(self, inputMatrix):
        """Predict function is used to propagate the inputs from 
        one layer to the next and get the final output from the layer"""
        # Pass on input of previous layer to the next
        # If 1st hidden layer, pass on the input
        layerOut = inputMatrix
        
        for layer in self.layers:
            layerOut = layer.forward(inputMatrix=layerOut)
            
        return layerOut
    
    def backprop(self, trainInput, trainOutput, learnRate, batchSize = 1, nIter = 100):
        """Back prop is used to update the weights based on the training sample
        It runs nIter iterations on the trainInput with batchSize number of rows
        Weight updates are carried out at learning rate learnRate"""
        
        if(not(isinstance(trainOutput,np.ndarray))):
            trainOutput = np.array(trainOutput)
        if(not(isinstance(trainInput,np.ndarray))):
            trainInput = np.array(trainInput)
        
        rows, columns = trainInput.shape
        
        for i in range(nIter):
            

            
            # Pick a random sample from the trainInput and trainOutput
            # Updated weights based on the same. Sample size is to be of size batchSize
            
            randomIndices = np.random.choice(range(rows),size=batchSize)
            
            # Sample from trainInput and trainOutput
            batchTrain = trainInput[randomIndices,:].reshape(batchSize,columns)
            batchTest = trainOutput[randomIndices,:]
        
            # A forward pass through the network
            self.predict(batchTrain)
            
            # Iterate backwards through the layers to pass the deltas
            delta = None
            
                        # Show progress every 50 iterations or so
            if i % 500 == 0:                
                print i
            
            for layer in self.layers[::-1]:
                
                delta = layer.backward(trainTarget=batchTest,delta=delta)
            
                layer.updateWeights(learnRate=learnRate)

In [31]:
mdata = mn(path='../datasets/MNIST/')

In [32]:
images,labels = mdata.load_training()
images = np.array(images)/255.
labels = np.array(labels).reshape(60000,1).astype(dtype = 'uint8')
labels = np.unpackbits(labels,axis=1)
labels = labels[:,4:8]

In [10]:
sampleIndices = np.random.choice(range(len(images)),size=5000)

In [11]:
imageTrain = images[sampleIndices]

In [12]:
labelTrain = labels[sampleIndices]

In [209]:
testImages,testLabels = mdata.load_testing()
testImages = np.array(testImages)/255.
testLabelsDigit = np.array(testLabels).reshape(10000,1).astype(dtype = 'uint8')

In [210]:
sampleTestIndices = np.random.choice(range(len(testImages)),size=1000) 

In [211]:
imageTest = testImages[sampleTestIndices]

In [212]:
labelTest = testLabelsDigit[sampleTestIndices]

In [348]:
# Training a neural net with 256 inputs, 300 hidden units (1 layer only)
nn1 = neuralNet(layerList=[784,300,4])

In [356]:
nn1.backprop(batchSize=20,learnRate=0.0001,nIter=10000,trainInput=imageTrain,trainOutput=labelTrain)

0
500
1000
1500
2000
2500
3000
3500
4000
4500
5000
5500
6000
6500
7000
7500
8000
8500
9000
9500


In [357]:
preds = nn1.predict(imageTest)

In [358]:
preds = np.array([np.round(arr) for arr in preds])

In [359]:
preds = np.append(np.zeros(4000).reshape(1000,4),preds,axis=1)

In [360]:
predDigits = np.packbits(preds.astype("bool"))

In [361]:
acc = [predDigits[i] == labelTest[i] for i in range(1000)]

In [362]:
print sum(acc)/1000.

[ 0.915]
